## Load data sets and split into features and labels

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.compose import ColumnTransformer

# dataset title: network intrusion detection
# dataset source: https://www.kaggle.com/datasets/sampadab17/network-intrusion-detection/data

nid_train_data = pd.read_csv('network_intrusion_detection_train.csv')
#print(nid_train_data.head())
print(nid_train_data.info())
nid_test_data = pd.read_csv('network_intrusion_detection_test.csv')
#print(nid_test_data.head())

# check how much of the traffic is malicious in the training data
class_distribution = nid_train_data['class'].value_counts()
print(class_distribution)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25192 entries, 0 to 25191
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     25192 non-null  int64  
 1   protocol_type                25192 non-null  object 
 2   service                      25192 non-null  object 
 3   flag                         25192 non-null  object 
 4   src_bytes                    25192 non-null  int64  
 5   dst_bytes                    25192 non-null  int64  
 6   land                         25192 non-null  int64  
 7   wrong_fragment               25192 non-null  int64  
 8   urgent                       25192 non-null  int64  
 9   hot                          25192 non-null  int64  
 10  num_failed_logins            25192 non-null  int64  
 11  logged_in                    25192 non-null  int64  
 12  num_compromised              25192 non-null  int64  
 13  root_shell      

In [3]:
label_encoder = LabelEncoder()
nid_train_data['class'] = label_encoder.fit_transform(nid_train_data['class'])

X_train = nid_train_data.drop(columns=['class'])  # X - features
y_train = nid_train_data['class']  # y - labels

In [4]:
# Identify categorical columns (protocol_type, service, flag) for encoding
categorical_cols = ['protocol_type', 'service', 'flag']
numeric_cols = X_train.columns.difference(categorical_cols)

# Use ColumnTransformer to one-hot encode categorical columns and leave numeric columns unchanged
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Preprocess both training and test sets
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(nid_test_data)

# Scale the data after encoding
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_preprocessed)
X_test_scaled = scaler.transform(X_test_preprocessed)

In [5]:
# Linear Perceptron
linear_perceptron = Perceptron(max_iter=1000)
linear_perceptron.fit(X_train_scaled, y_train)
lp_test_predictions = linear_perceptron.predict(X_test_scaled)

In [6]:
# Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train_scaled, y_train)
rf_test_predictions = random_forest.predict(X_test_scaled)

In [7]:
# Voting Classifier (combining RandomForest, DecisionTree, KNN, MLP)
decision_tree = DecisionTreeClassifier(random_state=42)
knn = KNeighborsClassifier(n_neighbors=5)
mlp = MLPClassifier(max_iter=300, random_state=42)

voting_clf = VotingClassifier(estimators=[
    ('rf', random_forest), 
    ('dt', decision_tree), 
    ('knn', knn), 
    ('mlp', mlp)],
    voting='hard')

voting_clf.fit(X_train_scaled, y_train)
vc_test_predictions = voting_clf.predict(X_test_scaled)

In [8]:
# predictions for test data
pd.DataFrame(lp_test_predictions, columns=['Linear_Perceptron_Predictions']).to_csv('lp_test_predictions.csv', index=False)
pd.DataFrame(rf_test_predictions, columns=['Random_Forest_Predictions']).to_csv('rf_test_predictions.csv', index=False)
pd.DataFrame(vc_test_predictions, columns=['Voting_Classifier_Predictions']).to_csv('vc_test_predictions.csv', index=False)
print("Predictions saved to CSV files. (done on test dataset)")

predictions = [
    (lp_test_predictions, 'Linear_Perceptron_Predictions', 'lp_test_predictions.csv'),
    (rf_test_predictions, 'Random_Forest_Predictions', 'rf_test_predictions.csv'),
    (vc_test_predictions, 'Voting_Classifier_Predictions', 'vc_test_predictions.csv')
]

# 0 for normal traffic, 1 for malcious traffic (abnormal)
for pred, col_name, file_name in predictions:
    pd.DataFrame(pred, columns=[col_name]).to_csv(file_name, index=False)
    pred_df = pd.read_csv(file_name)
    pred_count = pred_df[col_name].value_counts()
    print(f"Counts in {file_name}:")
    print(pred_count)
    print()

Predictions saved to CSV files. (done on test dataset)
Counts in lp_test_predictions.csv:
Linear_Perceptron_Predictions
1    13682
0     8862
Name: count, dtype: int64

Counts in rf_test_predictions.csv:
Random_Forest_Predictions
1    14196
0     8348
Name: count, dtype: int64

Counts in vc_test_predictions.csv:
Voting_Classifier_Predictions
1    13000
0     9544
Name: count, dtype: int64



In [9]:
# Evaluate accuracy on training data
lp_train_accuracy = linear_perceptron.score(X_train_scaled, y_train)
print(f'Linear Perceptron Training Accuracy: {lp_train_accuracy}')
from sklearn.model_selection import cross_val_score
# Perform cross-validation on the training data
lp_cv_scores = cross_val_score(linear_perceptron, X_train_scaled, y_train, cv=5)
print(f'Linear Perceptron Cross-Validation Accuracy: {lp_cv_scores.mean()}')


rf_train_accuracy = random_forest.score(X_train_scaled, y_train)
print(f'Random Forest Training Accuracy: {rf_train_accuracy}')
cv_scores = cross_val_score(random_forest, X_train_scaled, y_train, cv=5)
print(f'Random Forest Cross-Validation Accuracy: {cv_scores.mean()}')

vc_train_accuracy = voting_clf.score(X_train_scaled, y_train)
print(f'Voting Classifier Training Accuracy: {vc_train_accuracy}')
vc_cv_scores = cross_val_score(voting_clf, X_train_scaled, y_train, cv=5)
print(f'Voting Classifier Cross-Validation Accuracy: {vc_cv_scores.mean()}')

Linear Perceptron Training Accuracy: 0.9438710701810098
Linear Perceptron Cross-Validation Accuracy: 0.9682837582615818
Random Forest Training Accuracy: 0.9999603048586853
Random Forest Cross-Validation Accuracy: 0.997221316447076
Voting Classifier Training Accuracy: 0.9996824388694824
Voting Classifier Cross-Validation Accuracy: 0.9970228722514605


### IoT Intrusion dataset

In [10]:
IoT_data = pd.read_csv('IoT_Intrusion.csv')

#print(IoT_data.head())
#print(IoT_data.info())

print(IoT_data['label'].value_counts())

label
DDoS-ICMP_Flood            161281
DDoS-UDP_Flood             121205
DDoS-TCP_Flood             101293
DDoS-PSHACK_Flood           92395
DDoS-SYN_Flood              91644
DDoS-RSTFINFlood            90823
DDoS-SynonymousIP_Flood     80680
DoS-UDP_Flood               74787
DoS-TCP_Flood               59807
DoS-SYN_Flood               45207
BenignTraffic               24476
Mirai-greeth_flood          22115
Mirai-udpplain              20166
Mirai-greip_flood           16952
DDoS-ICMP_Fragmentation     10223
MITM-ArpSpoofing             7019
DDoS-ACK_Fragmentation       6431
DDoS-UDP_Fragmentation       6431
DNS_Spoofing                 4034
Recon-HostDiscovery          3007
Recon-OSScan                 2225
Recon-PortScan               1863
DoS-HTTP_Flood               1680
VulnerabilityScan             809
DDoS-HTTP_Flood               626
DDoS-SlowLoris                493
DictionaryBruteForce          324
BrowserHijacking              140
SqlInjection                  122
CommandI

In [11]:
# spliting dataset into training and test data

y = IoT_data.iloc[:, -1] # labels
X = IoT_data.iloc[:, :-1] # the rest of the dataset
 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

In [12]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
X_resampled, y_resampled = smote.fit_resample(X, y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
# before using Iot_Intrusion.csv to create new models, use the dataset to test previously created models with network_intrusion_detection.csv
lp_test_predictions = linear_perceptron.predict(X_test)
lp_train_accuracy = linear_perceptron.score(X_train, y_train)
print(f'Linear Perceptron Training Accuracy: {lp_train_accuracy}')


rf_test_predictions = random_forest.predict(X_train)
rf_train_accuracy = random_forest.score(X_train, y_train)
print(f'Random Forest Training Accuracy: {rf_train_accuracy}')

Linear Perceptron Training Accuracy: 0.7694938160931442
Random Forest Training Accuracy: 1.0


In [14]:
# Linear Perceptron
linear_perceptron = Perceptron(max_iter=1000)
linear_perceptron.fit(X_train, y_train)
lp_test_predictions = linear_perceptron.predict(X_test)

# Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)
rf_test_predictions = random_forest.predict(X_train)

# Voting Classifier (combining RandomForest, DecisionTree, KNN, MLP)
decision_tree = DecisionTreeClassifier(random_state=42)
knn = KNeighborsClassifier(n_neighbors=5)
mlp = MLPClassifier(max_iter=300, random_state=42)

voting_clf = VotingClassifier(estimators=[
    ('rf', random_forest), 
    ('dt', decision_tree), 
    ('knn', knn), 
    ('mlp', mlp)],
    voting='hard')

voting_clf.fit(X_train, y_train)
vc_test_predictions = voting_clf.predict(X_train)

KeyboardInterrupt: 

In [14]:
lp_train_accuracy = linear_perceptron.score(X_train, y_train)
print(f'Linear Perceptron Training Accuracy: {lp_train_accuracy}')
lp_cv_scores = cross_val_score(linear_perceptron, X_train, y_train, cv=5)
print(f'Linear Perceptron Cross-Validation Accuracy: {lp_cv_scores.mean()}')

rf_train_accuracy = random_forest.score(X_train, y_train)
print(f'Random Forest Training Accuracy: {rf_train_accuracy}')
cv_scores = cross_val_score(random_forest, X_train, y_train, cv=5)
print(f'Random Forest Cross-Validation Accuracy: {cv_scores.mean()}')

vc_train_accuracy = voting_clf.score(X_train, y_train)
print(f'Voting Classifier Training Accuracy: {vc_train_accuracy}')
vc_cv_scores = cross_val_score(voting_clf, X_train, y_train, cv=5)
print(f'Voting Classifier Cross-Validation Accuracy: {vc_cv_scores.mean()}')

Linear Perceptron Training Accuracy: 0.7694938160931442


KeyboardInterrupt: 